In [ ]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

In [ ]:
!helm install ../helm-charts/seldon-core --name seldon-core \
        --set cluster_manager.rbac=true \
        --set cluster_manager_client_secret=secret \
        --set cluster_manager_service_type=LoadBalancer \
        --set grafana_prom_service_type=LoadBalancer \
        --set apife_service_type=LoadBalancer

In [ ]:
from visualizer import get_graph

In [ ]:
import requests
from requests.auth import HTTPBasicAuth
from proto import prediction_pb2
from proto import prediction_pb2_grpc
import grpc
import commands

SELDON_API_IP=commands.getoutput("kubectl get svc seldon-apiserver -o jsonpath='{.status.loadBalancer.ingress[0].ip}'")

def get_token():
    payload = {'grant_type': 'client_credentials'}
    response = requests.post(
                "http://{}:8080/oauth/token".format(SELDON_API_IP),
                auth=HTTPBasicAuth('oauth-key', 'oauth-secret'),
                data=payload)
    token =  response.json()["access_token"]
    return token

def rest_request():
    token = get_token()
    headers = {'Authorization': 'Bearer '+token}
    payload = {"data":{"names":["a","b"],"tensor":{"shape":[2,2],"values":[0,0,1,1]}}}
    response = requests.post(
                "http://{}:8080/api/v0.1/predictions".format(SELDON_API_IP),
                headers=headers,
                json=payload)
    print response.text
    
def grpc_request():
    token = get_token()
    datadef = prediction_pb2.DefaultData(
            names = ["a","b"],
            tensor = prediction_pb2.Tensor(
                shape = [3,2],
                values = [1.0,1.0,2.0,3.0,4.0,5.0]
                )
            )
    request = prediction_pb2.SeldonMessage(data = datadef)
    channel = grpc.insecure_channel("{}:5000".format(SELDON_API_IP))
    stub = prediction_pb2_grpc.SeldonStub(channel)
    metadata = [('oauth_token', token)]
    response = stub.Predict(request=request,metadata=metadata)
    print response

In [ ]:
!kubectl get pods

In [ ]:
get_graph("resources/model.json")

In [ ]:
!kubectl apply -f resources/model.json

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath="{.status}"

In [ ]:
rest_request()

In [ ]:
grpc_request()

In [ ]:
!kubectl delete -f resources/model.json

In [ ]:
get_graph("resources/random_ab_test.json")

In [ ]:
!kubectl apply -f resources/random_ab_test.json

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

In [ ]:
rest_request()

In [ ]:
grpc_request()

In [ ]:
!kubectl delete -f resources/random_ab_test.json

In [ ]:
get_graph("resources/ensemble.json")

In [ ]:
!kubectl apply -f resources/ensemble.json

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

In [ ]:
rest_request()

In [ ]:
grpc_request()

In [ ]:
!kubectl delete -f resources/ensemble.json

In [ ]:
get_graph("resources/feature_transform.json")

In [ ]:
!kubectl apply -f resources/feature_transform.json

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

In [ ]:
rest_request()

In [ ]:
grpc_request()

In [ ]:
!kubectl delete -f resources/feature_transform.json

In [ ]:
get_graph("resources/complex_graph.json")

In [ ]:
!kubectl apply -f resources/complex_graph.json

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

In [ ]:
rest_request()

In [ ]:
grpc_request()

In [ ]:
!kubectl delete -f resources/complex_graph.json

In [ ]:
!helm delete seldon-core --purge